# Using CODEX Multiplexed Imaging Data as Input (No T cells)

This notebook was used to import CODEX multiplexed imaging data for the No T cell treated condition to initialize a tumor-tcell experiment

In [1]:
#Import Packages
#Needed for moving to output
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as pl
import itertools
from collections import Counter

#Import other needed packages
from vivarium.library.units import units, remove_units
from tumor_tcell.experiments.main import tumor_tcell_abm
from tumor_tcell.experiments.main import plots_suite
from tumor_tcell.experiments.main import get_tcells
from tumor_tcell.experiments.main import get_tumors
from tumor_tcell.library.phylogeny import get_phylogeny

Import the data needed from a CODEX multiplexed experiment (this experiment output only contains necessary input columns to the tumor-tcell)

In [3]:
#Read in the data from parent directory
analysis_dir = '/mnt/c/Users/akoya-stanford/Python_Code/tumor-tcell/out/analysis/'
experiment_id = 'Cntrl/'
experiment_dir = analysis_dir+experiment_id
os.chdir(experiment_dir)

#Read in the data from parent directory
df_2HC_exp = pd.read_csv('Cntrl_exp_1of4.csv')
df_2HC_exp.rename(columns = {"Unnamed: 0": "cell_index"},  inplace = True) 
df_2HC_exp.set_index('cell_index', inplace = True)
df_2HC_exp

'/mnt/c/Users/akoya-stanford/Python_Code/tumor-tcell/out/analysis/Cntrl'

This block of code establishes intial parameters for the tumor_tcell_abm simulation:
* bounds - the size of the experimental space in microns for x,y
* Y pos - the column number for the y column in dataframe imported
* X pos - the column number for the x column in dataframe imported
* H2Kb pos - the column number for the H2Kb/MHCI column in dataframe imported (used for tumor cells)
* PD1 - the column number for the PD1 column in dataframe imported (used for T cells)
* PDL1 - the column number for the PD1 column in dataframe imported (used for tumor cells)
* Celltype - the column number for the cell type identified from the CODEX data
* IFNg_con - the column number for thedefault level of IFNg for all cells
* tumor_diameter - control tumor diameter in simulation
* tcell_diameter - control t cell diameter in simulation
It will loop through the imported dataframe and establish a structure of dictionaries that can be used as input to the tumor_tcell_abm simulation 

In [5]:
#Set bounds
bounds = [1800,1800]

#Create a copy
df_2HC_exp1 = df_2HC_exp.copy()

#Column Number for iloc iterating
Ypos = 0
Xpos = 1
H2Kb = 3
PD1 = 4
PDL1 = 5
Celltype = 7
IFNg_con = 8 

tumor_diameter = 15
tcell_diameter = 7.5

tumors = {}
IFNg = {}
tcells = {}

for i in range(len(df_2HC_exp1)):
  if df_2HC_exp1.iloc[i, Celltype] == 'PDL1p' or df_2HC_exp1.iloc[i, Celltype] == 'PDL1n':
    tumors.update({
        'tumor_'+ str(df_2HC_exp1.index[i]) : {
            'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
            'type': 'tumor',
            'cell_state': df_2HC_exp1.iloc[i, Celltype],
            'PDL1': df_2HC_exp1.iloc[i, PDL1],
            'MHCI': df_2HC_exp1.iloc[i, H2Kb],
            'diameter': tumor_diameter*units.um,
        }
    })
    
    IFNg.update({
        df_2HC_exp1.index[i] : {
            'amount': df_2HC_exp1.iloc[i, IFNg_con],
            'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
        }
    })

  else: 
    tcells.update({
        'tcell_'+ str(df_2HC_exp1.index[i]) : {
            'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
            'type': 'tcell',
            'cell_state': df_2HC_exp1.iloc[i, Celltype],
            'PD1': df_2HC_exp1.iloc[i, PD1],
            'diameter': tcell_diameter*units.um,
        }
    })
    IFNg.update({
        df_2HC_exp1.index[i]: {
        'amount': df_2HC_exp1.iloc[i, IFNg_con],
        'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
        }
    })

tcells

{}

Here tumor_tcell_abm can directly be called, initialized with the two dictionaries created (tumors, tcells), set for the time and bounds. 
Also, parameters like halt_threshold (number of cells to stop the simulation) and emit_step (how often to output data from the simulation in second). 
emitter=database enables storing the data in a database for future retrieval and usually is not changed. 

In [6]:
#using experimental CODEX data
data = tumor_tcell_abm(tumors=tumors, 
                       tcells=tcells, 
                       n_tcells = 0,
                       total_time=259200, 
                       bounds=[b*units.um for b in bounds], 
                       halt_threshold=10000,
                       emit_step=600,
                       emitter='database',)
data;

Initializing experiment tumor_tcell_20210716.142727


 69%|██████▉   | 299/432 [53:18:30<55:17:58, 1496.83s/it]

halt threshold of 10000 reached at time = 178800


 69%|██████▉   | 300/432 [53:18:30<38:25:20, 1047.89s/it]

halt threshold of 10000 reached at time = 179400


...

halt threshold of 10000 reached at time = 257400


100%|█████████▉| 431/432 [53:19:23<00:00,  2.59it/s]

halt threshold of 10000 reached at time = 258000


100%|██████████| 432/432 [53:19:24<00:00, 444.36s/it]

halt threshold of 10000 reached at time = 258600
Completed in 191964.29 seconds
